In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import os

In [3]:
import torch 
import random
torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)

#### Prepare the files for training:

In [3]:
df_annotation = pd.read_pickle('../data/annotation-results/MACE-measure/final_mace_predictions.pkl')
df_annotation['turn_text'] = df_annotation.apply(lambda row: {
                                                            'author': row['turn_text']['author'], 
                                                            'text' : row['topic'].replace('_', ' ') + ' [SEP] ' +  row['turn_text']['text']
                                                           } ,axis=1)

# add row_num
df_annotation.insert(loc=0,column='row_num',value=np.arange(len(df_annotation)))

In [4]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=123)

In [5]:
def prepare_data_files(df, out_dir, target_clm, count_validation_topics:int=1, random_validate_topic:bool=True):
    # adds training, validation and testing dataframes into the folder made on the name of test_topic

    topic_list = df.topic.unique()
    print(topic_list)
    for test_topic in topic_list:
        
        train_list = list(set(topic_list) - set([test_topic]))
        train_topics, validate_topics = train_test_split(train_list, test_size=count_validation_topics, shuffle=random_validate_topic)

        # saving the test topics in the folder with name test_topic
        test_df     = df[df.topic==test_topic]
        validate_df = df[df.topic.isin(validate_topics)]
        train_df    = df[df.topic.isin(train_topics)]
        
        #balance the data
        new_training_df, y = ros.fit_resample(train_df, train_df[target_clm])
        new_training_df[target_clm] = y
        
        print(len(new_training_df))
        test_topic_act_df = test_df[['row_num','topic_func_label','turn_text']]
        test_dlg_act_df = test_df[['row_num','dlg_act_label','turn_text']]
        test_exp_act_df = test_df[['row_num','exp_act_label','turn_text']]

        validate_topic_act_df = validate_df[['row_num','topic_func_label','turn_text']]
        validate_dlg_act_df = validate_df[['row_num','dlg_act_label','turn_text']]
        validate_exp_act_df = validate_df[['row_num','exp_act_label','turn_text']]

        train_topic_act_df = new_training_df[['row_num','topic_func_label','turn_text']]
        train_dlg_act_df = new_training_df[['row_num','dlg_act_label','turn_text']]
        train_exp_act_df = new_training_df[['row_num','exp_act_label','turn_text']]

        # save it into test folder
        # check if the folder by that name exist
        # multi_task_learning -> explanantion_assessment/data/test_topic_folders
        if not os.path.isdir(out_dir+test_topic):
            os.mkdir(out_dir+test_topic)

        test_topic_act_df.to_csv(out_dir+test_topic+'/test_topic_act.tsv', sep='\t', header=False, index=False)
        test_dlg_act_df.to_csv(out_dir+test_topic+'/test_dlg_act.tsv', sep='\t', header=False, index=False)
        test_exp_act_df.to_csv(out_dir+test_topic+'/test_exp_act.tsv', sep='\t', header=False, index=False)

        validate_topic_act_df.to_csv(out_dir+test_topic+'/validate_topic_act.tsv', sep='\t', header=False,index=False)
        validate_dlg_act_df.to_csv(out_dir+test_topic+'/validate_dlg_act.tsv', sep='\t', header=False, index=False)
        validate_exp_act_df.to_csv(out_dir+test_topic+'/validate_exp_act.tsv', sep='\t', header=False, index=False)
            

        train_topic_act_df.to_csv(out_dir+test_topic+'/train_topic_act.tsv', sep='\t', header=False, index=False)
        train_dlg_act_df.to_csv(out_dir+test_topic+'/train_dlg_act.tsv', sep='\t', header=False, index=False)
        train_exp_act_df.to_csv(out_dir+test_topic+'/train_exp_act.tsv', sep='\t', header=False,index=False)

In [6]:
prepare_data_files(df_annotation, './topic-func-data-and-models/', 'topic_func_label')

['virtual_reality' 'sleep_scientist' 'origani' 'nano_technology'
 'music_harmony' 'machine_learning' 'lasers' 'hacking' 'gravity'
 'dimensions' 'connectome' 'blockchain' 'blackhole']
2696
2652
2504
2688
2432
2680
2708
2832
2604
2412
2840
2840
2832


In [7]:
prepare_data_files(df_annotation, './dlg-act-data-and-models/', 'dlg_act_label')

['virtual_reality' 'sleep_scientist' 'origani' 'nano_technology'
 'music_harmony' 'machine_learning' 'lasers' 'hacking' 'gravity'
 'dimensions' 'connectome' 'blockchain' 'blackhole']
5950
6070
5520
6260
6010
5810
6010
5680
5720
5740
6450
6070
6360


In [8]:
prepare_data_files(df_annotation, './exp-act-data-and-models/', 'exp_act_label')

['virtual_reality' 'sleep_scientist' 'origani' 'nano_technology'
 'music_harmony' 'machine_learning' 'lasers' 'hacking' 'gravity'
 'dimensions' 'connectome' 'blockchain' 'blackhole']
6180
5620
5840
6040
5780
6020
5930
5960
5360
4940
5700
5620
5670


### Train 13 models, one for each topic:

#### Train on topic func:

In [12]:
!sh preprocess_data.sh ./topic-func-data-and-models/

2022-05-09 13:30:22.359867: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
task object created from task file...
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a

In [ ]:
!sh train-models.sh ./topic-func-data-and-models

#### Train on dialogue act:

In [10]:
!sh preprocess_data.sh ./dlg-act-data-and-models/

2022-05-09 13:06:52.356108: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
task object created from task file...
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a

In [ ]:
!sh train-models.sh ./dlg-act-data-and-models/

training on virtual_reality in ./dlg-act-data-and-models/
2022-05-07 10:01:05.471688: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
128
INFO - logger created.
INFO - ARGS : {'data_dir': './dlg-act-data-and-models/virtual_reality/bert-base-uncased_prepared_data', 'task_file': './task.yml', 'out_dir': './dlg-act-data-and-models/virtual_reality/model', 'epochs': 15, 'freeze_shared_model': False, 'train_batch_size': 4, 'eval_batch_size': 8, 'grad_accumulation_steps': 2, 'num_of_warmup_steps': 0, 'learning_rate': 2e-05, 'epsilon': 1e-08, 'grad_clip_value': 1.0, 'log_file': 'multi_task_logs.log', 'log_per_updates': 25, 'seed': 42, 'max_seq_len': 128, 'save_per_updates': 1000, 'limit_save': 10, 'load_saved_model': None, 'eval_while_train': True, 'test_while_train': False, 'resume_train': False, 'finetune': False, 'debug_mode': False, 'silent': False}
INFO - Task params object created from task file...
INFO - task paramete

#### Training on explainability act:

In [11]:
!sh preprocess_data.sh ./exp-act-data-and-models/

2022-05-09 13:11:02.967683: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
task object created from task file...
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a

In [ ]:
!sh train-models.sh ./exp-act-data-and-models/

training on virtual_reality in ./exp-act-data-and-models/
2022-05-08 14:50:23.018608: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
INFO - logger created.
INFO - ARGS : {'data_dir': './exp-act-data-and-models/virtual_reality/bert-base-uncased_prepared_data', 'task_file': './task.yml', 'out_dir': './exp-act-data-and-models/virtual_reality/model', 'epochs': 15, 'freeze_shared_model': False, 'train_batch_size': 4, 'eval_batch_size': 8, 'grad_accumulation_steps': 2, 'num_of_warmup_steps': 0, 'learning_rate': 2e-05, 'epsilon': 1e-08, 'grad_clip_value': 1.0, 'log_file': 'multi_task_logs.log', 'log_dir': '/var/tmp/./exp-act-data-and-models//virtual_reality', 'log_per_updates': 25, 'seed': 42, 'max_seq_len': 128, 'save_per_updates': 0, 'limit_save': 10, 'load_saved_model': None, 'eval_while_train': False, 'test_while_train': False, 'resume_train': False, 'finetune': False, 'debug_mode': False, 'silent': False}
INFO - Task 

In [7]:
print('Doneeee')

Doneeee


### Perform prediction:

In [4]:
%load_ext autoreload

In [5]:
import sys
import glob
import json
sys.path.append('../../multi-task-NLP/')
from sklearn.metrics import f1_score, precision_score, recall_score

In [6]:
%autoreload
from infer_pipeline import inferPipeline

In [7]:
df_annotation = pd.read_pickle('../data/annotation-results/MACE-measure/final_mace_predictions.pkl')
df_annotation['turn_text'] = df_annotation.apply(lambda row: {
                                                            'author': row['turn_text']['author'], 
                                                            'text' : row['topic'].replace('_', ' ') + ' [SEP] ' +  row['turn_text']['text']
                                                           } ,axis=1)

df_annotation.insert(loc=0,column='row_num',value=np.arange(len(df_annotation)))

In [8]:
def find_best_model_path(model_dir, topic_content, task):
    eval_epochs = topic_content.split('********** {} Evaluation************'.format(task))
    eval_epochs = [float(run.split('\n')[3].replace("INFO - classification_f1_score : ", "")) for run in eval_epochs[1:]]
    #print(eval_epochs)
    best_epoch   = np.argmax(eval_epochs)
    #print(model_dir + "/model/multi_task_model_{}_*".format(best_epoch))
    epoch_models = list(glob.glob(model_dir + "/model/multi_task_model_{}_*".format(best_epoch)))
    if len(epoch_models) == 0 :
        print('Didnt find best model...')
        epoch_models = list(glob.glob(model_dir + "/model/multi_task_model_*"))
    
    return max(epoch_models, key=os.path.getctime)

def load_topic_logs(working_dir):
    topic_logs = {}
    for f in glob.glob(working_dir + '/logs/*/multi_task_logs.log'):
        content = open(f).read()
        topic = f.split('/')[-2]
        topic_logs[topic] = content
    
    return topic_logs
    
def bert_pred_labels(df, label, working_dir, task_name):
    
    df['{}_mt_pred'.format(label)] = [None] * len(df)
    
    topic_logs = load_topic_logs(working_dir)
    
    topic_list = df.topic.unique()
    for test_topic in topic_list:
        print(test_topic)
        inputs = [[x['text']] for x in df[df.topic==test_topic]['turn_text'].tolist()]
        
        #find best model on validation
        #Note that all could have been avoided if we named the log_dir at the time of training, but had not time to retrain the models..
        model_path = find_best_model_path(working_dir + "/" + test_topic, topic_logs[test_topic], task_name)
        #epoch_models = list(glob.glob(working_dir + "/" + test_topic + "/model/multi_task_model_*"))
        #model_path   = min(epoch_models, key=os.path.getctime)
        print(model_path)
        pipe  = inferPipeline(modelPath = model_path, maxSeqLen = 128)
        preds = pipe.infer(inputs, [task_name]) 
        df.loc[df.topic == test_topic, '{}_mt_pred'.format(label)] = [x[task_name][0] for x in preds]
        #print(preds)
        
    return df

In [9]:
df_annotation = bert_pred_labels(df_annotation, 'topic-func-lable', './topic-func-data-and-models', 'TaskA')

virtual_reality
./topic-func-data-and-models/virtual_reality/model/multi_task_model_4_2527.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

sleep_scientist
./topic-func-data-and-models/sleep_scientist/model/multi_task_model_1_996.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

origani
./topic-func-data-and-models/origani/model/multi_task_model_8_4225.pt


...

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

blockchain
./topic-func-data-and-models/blockchain/model/multi_task_model_6_3727.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

blackhole
./topic-func-data-and-models/blackhole/model/multi_task_model_9_5310.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

In [10]:
df_annotation = bert_pred_labels(df_annotation, 'dlg-act-lable', './dlg-act-data-and-models', 'TaskB')

virtual_reality
./dlg-act-data-and-models/virtual_reality/model/multi_task_model_3_4464.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

sleep_scientist
./dlg-act-data-and-models/sleep_scientist/model/multi_task_model_7_9108.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

origani
./dlg-act-data-and-models/origani/model/multi_task_model_9_10350.pt


...

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

blockchain
./dlg-act-data-and-models/blockchain/model/multi_task_model_9_11385.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

blackhole
./dlg-act-data-and-models/blackhole/model/multi_task_model_6_8347.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

In [12]:
df_annotation = bert_pred_labels(df_annotation, 'exp-act-lable', './exp-act-data-and-models', 'TaskC')

virtual_reality
./exp-act-data-and-models/virtual_reality/model/multi_task_model_9_11595.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

sleep_scientist
./exp-act-data-and-models/sleep_scientist/model/multi_task_model_9_10545.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

origani
./exp-act-data-and-models/origani/model/multi_task_model_9_10950.pt


...

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

blockchain
./exp-act-data-and-models/blockchain/model/multi_task_model_9_10545.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

blackhole
./exp-act-data-and-models/blackhole/model/multi_task_model_7_8508.pt


loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transfo

In [13]:
df_annotation.to_pickle('./mt_final_preds.pkl')

In [14]:
df_annotation.head(n=20)

,row_num,task_id,turn_id,topic_func_label,dlg_act_label,exp_act_label,topic,lvl,turn_text,topic-func-lable_mt_pred,dlg-act-lable_mt_pred,exp-act-lable_mt_pred
0,0,142,1,(T04) Other - No topic was introduced,(D09) To provide informing statement,(E02) Testing prior knowledge,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'virtual reali...",(T01) It is the main topic,(D09) To provide informing statement,(E03) Provide an explanation
1,1,142,2,(T01) It is the main topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainee', 'text': 'virtual reali...",(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation
2,2,142,3,(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'virtual reali...",(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation
3,3,142,4,(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainee', 'text': 'virtual reali...",(T01) It is the main topic,(D09) To provide informing statement,(E03) Provide an explanation
4,4,142,5,(T01) It is the main topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'virtual reali...",(T02) A subtopic,(D09) To provide informing statement,(E03) Provide an explanation
5,5,142,6,(T04) Other - No topic was introduced,(D07) To provide agreement statement,(E05) Signaling understanding,virtual_reality,colleague,"{'author': 'Explainee', 'text': 'virtual reali...",(T04) Other - No topic was introduced,(D10) Other,(E05) Signaling understanding
6,6,142,7,(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'virtual reali...",(T02) A subtopic,(D09) To provide informing statement,(E03) Provide an explanation
7,7,142,8,(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainee', 'text': 'virtual reali...",(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation
8,8,142,9,(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'virtual reali...",(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation
9,9,142,10,(T04) Other - No topic was introduced,(D07) To provide agreement statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainee', 'text': 'virtual reali...",(T03) A related topic,(D09) To provide informing statement,(E03) Provide an explanation


In [15]:
f1_score(df_annotation.topic_func_label.tolist(), df_annotation['topic-func-lable_mt_pred'].tolist(), average='macro')

0.3949154959488238

In [16]:
f1_score(df_annotation.dlg_act_label.tolist(), df_annotation['dlg-act-lable_mt_pred'].tolist(), average='macro')

0.26158331871483187

In [17]:
f1_score(df_annotation.exp_act_label.tolist(), df_annotation['exp-act-lable_mt_pred'].tolist(), average='macro')

0.21305796862584137